In [1]:
from scHash import *
from util import *
from dataModule import *
import anndata as ad

# Pancreas Dataset

We demonstrate how scHash encode multiple datasets into hash codes for six public avaialble Pancreas datasets.

The raw data for first five datasets can be obtained from Harmony https://github.com/immunogenomics/harmony2019/tree/master/data/figure5

The sixth Pancreas dataset is public available at GSE83139.

We compiled the six datasets into one AnnData object for easy demonstration. The processed data can be downloaded hereh.

In [6]:
data_dir = '../../../share_data/Pancreas_Wang/fivepancreas_wang_raw.h5ad'

# This data contains both the reference and query source
data = ad.read_h5ad(data_dir)

# Split the reference and query data. We use Muraro(celseq) for query as demonstration. 
# You could specify your query datasets.
train = data[data.obs.dataset != 'smartseq']
test = data[data.obs.dataset == 'smartseq']

# set up datamodule
########## consider write this into one function as setup_data
datamodule = Cross_DataModule(train_data = train, cell_type_key='cell_type')
datamodule.setup()
N_CLASS = datamodule.N_CLASS
N_FEATURES = datamodule.N_FEATURES

# set the query data
# this can be also set after train
datamodule.test_data = test


########### consider write into a function again
# Init ModelCheckpoint callback
checkpointPath = '../checkpoint/'

# Train
checkpoint_callback = ModelCheckpoint(
                            monitor='Val_F1_score_median_CHC_epoch',
                            dirpath=checkpointPath,
                            filename='scHash-{epoch:02d}-{Val_F1_score_median_CHC_epoch:.3f}',
                            verbose=True,
                            # save_last = True,
                            mode='max'
                            )
early_stopping_callback = EarlyStopping(monitor="Val_F1_score_median_CHC_epoch")
start = time.time()
trainer = pl.Trainer(max_epochs=200,
                    gpus=1,
                    check_val_every_n_epoch=10,
                    progress_bar_refresh_rate=50,
                    callbacks=[checkpoint_callback]
                    )
print("Number of Feature: ", N_FEATURES)
model = scHashModel(N_CLASS, N_FEATURES)

trainer.fit(model = model, datamodule = datamodule)



# Test the best model
best_model_path = checkpoint_callback.best_model_path
best_model = scHashModel.load_from_checkpoint(best_model_path, n_class=N_CLASS, n_features=N_FEATURES)

best_model.eval()

start = time.time()
trainer.test(model=best_model, datamodule=datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of Feature:  1000
hparam: l_r = 1.2e-05, lambda = 0.001, beta = 0.9999


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | hash_layer | Sequential | 641 K 
------------------------------------------
641 K     Trainable params
0         Non-trainable params
641 K     Total params
2.567     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

################### compute_result time:  0.8244278430938721
################### cell_assign time:  0.05458855628967285
################### query time:  0.8790163993835449


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          Test_F1          │    0.6214442449078369     │
│      Test_F1_Median       │    0.45772854596384005    │
│       Test_accuracy       │    0.5864118933677673     │
│   Test_cell_assign_time   │    0.05458855628967285    │
│     Test_hashing_time     │    0.8244278430938721     │
│      Test_precision       │    0.7585122142303424     │
│      Test_query_time      │    0.8790163993835449     │
│        Test_recall        │     0.586411889596603     │
└───────────────────────────┴───────────────────────────┘

[{'Test_F1': 0.6214442449078369,
  'Test_F1_Median': 0.45772854596384005,
  'Test_precision': 0.7585122142303424,
  'Test_recall': 0.586411889596603,
  'Test_hashing_time': 0.8244278430938721,
  'Test_cell_assign_time': 0.05458855628967285,
  'Test_query_time': 0.8790163993835449,
  'Test_accuracy': 0.5864118933677673}]